In [1]:
!pip install pycountry
!pip install pandas
!pip install autocorrect

In [16]:
import pandas as pd
import numpy as np
import pycountry
import re

In [17]:
#data = pd.read_csv("/Users/davidcastrejon/Downloads/candyhierarchy2017.csv", encoding="latin")
data = pd.read_csv("candyhierarchy2017.csv", encoding="latin")
data.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


# Column 1

In [18]:
COLUMN_NAME = "Q1: GOING OUT?"
# convert to lowercase
data[COLUMN_NAME] = data[COLUMN_NAME].str.lower()
# Get count of yes, no, nan.
colOneValuesCount = data[COLUMN_NAME].astype(str).value_counts(dropna=False)
yesCountPercentage = colOneValuesCount.get("yes", 0) / (data[COLUMN_NAME].size - colOneValuesCount.get("nan", 0))
yesCountFill = round(yesCountPercentage * colOneValuesCount.get("nan", 0))
# shuffle the data frame 
data = data.sample(frac=1).reset_index(drop=True)
   # fill our nan values with yes according to the countFill
try:
    data[COLUMN_NAME].fillna("yes", limit=yesCountFill, inplace=True)
    data[COLUMN_NAME].fillna("no", inplace=True)
    data[COLUMN_NAME].unique()
except:
    pass
# colOneValuesCount

# we can use regex validation for an extra layer of security (good for data integrity for large data sets)
 
# define reg pattern to use
regexPattern = r"^(yes|no)$" 

# parse through data to check if answers match regexPattern
def process_going_out(going_out):
    if re.match(regexPattern, going_out):
        return going_out
    else:
        return 'no'

data[COLUMN_NAME] = data[COLUMN_NAME].apply(process_going_out)

print(data[COLUMN_NAME].unique())


['no' 'yes']


## Column 2

In [19]:
COLUMN_NAME = "Q2: GENDER"

data[COLUMN_NAME] = data[COLUMN_NAME].str.lower()
# data[COLUMN_NAME].unique() # checked unique values, only 'male', 'female', 'I'd rather not say', 'Other'
data[COLUMN_NAME].astype(str).fillna("Other", inplace=True)  
data[COLUMN_NAME].tail(2)


# we can use regex validation for an extra layer of security (good for data integrity for large data sets)

# define reg pattern to use
regexPattern = r"^(female|male|other|i'd rather not say)$" 

# parse through data to check if answers match regexPattern (as stated by slides, NIL values = 'other')
def process_gender(gender):
    gender_str = str(gender)
    match = re.match(regexPattern, gender_str)
    if match:
        return match.group()
    else:
        return 'other'
    
# apply column changes and print
data[COLUMN_NAME] = data[COLUMN_NAME].apply(process_gender)
print(data[COLUMN_NAME])


0         male
1       female
2       female
3         male
4       female
         ...  
2455      male
2456    female
2457      male
2458      male
2459    female
Name: Q2: GENDER, Length: 2460, dtype: object


# Column 3

In [20]:
COLUMN_NAME = "Q3: AGE"
# Unique
#for the text data in the data frame that is age we can use reguaar expression to find the numbers and then convert them to numeric values
#
# 
"""
[nan, '44', '49', '40', '23', '53', '33', '43', '56', '64', '37',
       '59', '48', '54', '36', '45', '25', '34', '35', '38', '58', '50',
       '47', '16', '52', '63', '65', '41', '27', '31', '61', '46', '42',
       '62', '29', '39', '32', '28', '69', '67', '30', '22', '26', '51',
       '70', '24', '18', '19', 'Old enough', '57', '60', '66', '12',
       'Many', '55', '72', '?', '21', '11', 'no', '9', '68', '20', '6',
       '10', '71', '90', '13', '45-55', '312', '99', '7', 'hahahahaha',
       '88', '39.4', '74', '102', 'older than dirt', '17', '15', '8',
       '75', '5u', 'Enough', 'See question 2', '24-50', '14', 'Over 50',
       '100', '76', '1000', 'sixty-nine', '46 Halloweens.', 'ancient',
       '77', 'OLD', 'old', '73', '70 1/2', '1', 'MY NAME JEFF', '4',
       '59 on the day after Halloween', 'old enough', 'your mom',
       'I can remember when Java was a cool new language', '60+']
"""

# Replace series as a numeric type rounded to the nearest whole number.
data[COLUMN_NAME] = pd.to_numeric(data[COLUMN_NAME], errors='coerce').round() # convert to numeric values & round

# remove columns which have values not in range 5-100
data[COLUMN_NAME] = data[COLUMN_NAME][(data[COLUMN_NAME] >= 5) & (data[COLUMN_NAME] <= 100)]  # filter to only ages of 5-100
# subsetcolumn label or sequence of labels, optional
data.dropna(subset=[COLUMN_NAME], inplace=True) # drop all rows with nan values
data[COLUMN_NAME]


# double check if there are any non-numeric values, this should be empty:
non_numeric_values = data[~data[COLUMN_NAME].apply(lambda x: np.isreal(x))]
message = "number of non-numeric values in column " + COLUMN_NAME + " (should be empty):"
print(message, non_numeric_values)

# fill nil values with the median of the age column (or mean if you want, i commented that out)
data[COLUMN_NAME].fillna(data[COLUMN_NAME].median(), inplace=True)
# data[COLUMN_NAME].fillna(data[COLUMN_NAME].mean(), inplace=True)

# double check to see if any nulls are left
message = "number of nulls in column " + COLUMN_NAME + " (should be 0):"
print (message, data[COLUMN_NAME].isna().sum())

number of non-numeric values in column Q3: AGE (should be empty): Empty DataFrame
Columns: [Internal ID, Q1: GOING OUT?, Q2: GENDER, Q3: AGE, Q4: COUNTRY, Q5: STATE, PROVINCE, COUNTY, ETC, Q6 | 100 Grand Bar, Q6 | Anonymous brown globs that come in black and orange wrappers	(a.k.a. Mary Janes), Q6 | Any full-sized candy bar, Q6 | Black Jacks, Q6 | Bonkers (the candy), Q6 | Bonkers (the board game), Q6 | Bottle Caps, Q6 | Box'o'Raisins, Q6 | Broken glow stick, Q6 | Butterfinger, Q6 | Cadbury Creme Eggs, Q6 | Candy Corn, Q6 | Candy that is clearly just the stuff given out for free at restaurants, Q6 | Caramellos, Q6 | Cash, or other forms of legal tender, Q6 | Chardonnay, Q6 | Chick-o-Sticks (we donÕt know what that is), Q6 | Chiclets, Q6 | Coffee Crisp, Q6 | Creepy Religious comics/Chick Tracts, Q6 | Dental paraphenalia, Q6 | Dots, Q6 | Dove Bars, Q6 | Fuzzy Peaches, Q6 | Generic Brand Acetaminophen, Q6 | Glow sticks, Q6 | Goo Goo Clusters, Q6 | Good N' Plenty, Q6 | Gum from baseball ca

# Column 4


In [21]:
# use the pycountry and autocorrect libraries
import pycountry
from autocorrect import Speller
COLUMN_NAME = "Q4: COUNTRY"

# initialize Speller obj from library
spell = Speller() 

# function to autocorrect country names
def autocorrectCountries(value):
    value=value.lower()
    orig=value
    corrected_country = spell(value) # autocorrect name
    try:
        country = pycountry.countries.lookup(corrected_country) # look up country from pycountry
        if country:
            return country.name.lower() # if found, return the (lowercased) standard spelling
    except Exception as e:
        return orig # else return nan (not sure if there's something better?)

# apply to each country in column and retrieve all the unique names
data[COLUMN_NAME] = data[COLUMN_NAME].astype(str).apply(autocorrectCountries) 
# data[COLUMN_NAME].unique()

def standardize_country_name(name):
    name = name.lower().strip()

    # Remapping for United States
    if any(keyword in name for keyword in ['us', 'u.s.', 'america', 'states', 'us of a', 'unhinged states', 'usa ', 'united state', 'unite states', "'merica", 'usausausa', 'u s ', 'u.s. ', 'u s a', "usa? hard to tell anymore..", "i don't know anymore", 'north carolina', 'new york', 'california', 'pittsburgh', 'new jersey', 'n. america', 'usa usa usa!!!!', 'usa! usa! usa!', 'murica', 'murrika']):
        return 'united states'

    # Remapping for United Kingdom
    if any(keyword in name for keyword in ['uk', 'u.k.', 'england', 'united kingdom', 'scotland', 'uk ', 'u.k. ', 'endland']):
        return 'united kingdom'

    # Remapping for Canada
    if any(keyword in name for keyword in ['canada', 'soviet canuckistan', 'i pretend to be from canada, but i am really from the united states.', 'canada ', 'canae', 'canada`']):
        return 'canada'

    # Remapping for United Arab Emirates
    if 'united arab emirates' in name:
        return 'united arab emirates'

    # Remapping for China
    if any(keyword in name for keyword in ['china', 'taiwan, province of china', 'hong kong']):
        return 'china'

    # ... additional remapping rules
    if 'mexico' in name:
        return 'mexico'

    if 'iceland' in name:
        return 'iceland'

    if 'germany' in name:
        return 'germany'

    if any(keyword in name for keyword in ['netherlands', 'the netherlands']):
        return 'netherlands'

    if 'denmark' in name:
        return 'denmark'

    if any(keyword in name for keyword in ['ireland', 'ireland ']):
        return 'ireland'

    if 'indonesia' in name:
        return 'indonesia'

    if any(keyword in name for keyword in ['japan', 'korea', 'korea, republic of']):
        return 'japan/korea'

    if 'europe' in name:
        return 'europe'

    if 'switzerland' in name:
        return 'switzerland'

    if 'spain' in name:
        return 'spain'

    if any(keyword in name for keyword in ['france', 'france ']):
        return 'france'

    if 'sweden' in name:
        return 'sweden'

    if 'trumpistan' in name:
        return 'fictional/cultural reference'

    if 'finland' in name:
        return 'finland'

    if 'greece' in name:
        return 'greece'

    if 'south africa' in name:
        return 'south africa'

    if 'costa rica' in name:
        return 'costa rica'

    if 'singapore' in name:
        return 'singapore'

    if 'earth' in name:
        return 'earth'

    if 'narnia' in name or 'cascadia' in name or 'ud' in name:
        return 'fictional/cultural reference'

    if 'nan' in name:
        return 'unknown'

    # If name doesn't match any known patterns, keep it as is
    return 'other'

COLUMN_NAME = "Q4: COUNTRY"
data[COLUMN_NAME] = data[COLUMN_NAME].astype(str).apply(standardize_country_name)
data[COLUMN_NAME].value_counts()


Q4: COUNTRY
united states                   2014
canada                           223
united kingdom                    39
germany                           10
japan/korea                        7
netherlands                        7
china                              5
unknown                            5
ireland                            4
other                              4
mexico                             4
fictional/cultural reference       4
france                             3
switzerland                        3
denmark                            2
sweden                             2
iceland                            1
greece                             1
europe                             1
spain                              1
south africa                       1
singapore                          1
indonesia                          1
finland                            1
costa rica                         1
earth                              1
Name: count, dtype: int64

In [22]:
print(data[COLUMN_NAME].count())

2346


# Column 5

- ### Oppenheimer this column 💣

In [23]:
data.drop(columns=['Q5: STATE, PROVINCE, COUNTY, ETC'],inplace=True)

In [24]:
data.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90276906,no,male,41.0,united states,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90302444,no,female,55.0,united states,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90277936,no,female,39.0,united states,MEH,MEH,JOY,MEH,MEH,...,Pepsi.,The suggestion of sugar overload here has caus...,Blue and black,NaN,Friday,NaN,1.0,NaN,NaN,"(63, 30)"
3,90273058,no,male,42.0,united states,JOY,MEH,JOY,DESPAIR,DESPAIR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90285361,no,female,56.0,united states,JOY,DESPAIR,JOY,JOY,MEH,...,pennies,NaN,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(68, 5)"


# Columns 5 - 106

In [25]:
start_index = 5
end_index = 106

# Use list slicing to extract column names into a list
sequential_column_names = data.columns[start_index:end_index + 1].tolist()

for col in sequential_column_names:
    # Use the 'replace' method to map values to '001', '010', '100'; Applying Hot Encoding
    data[col] = data[col].replace({'JOY': '00001', 'MEH': '0010', 'DESPAIR': '0100', np.nan: '1000'})
    data[col] = data[col].replace({'00001': 0, '0010': 1, '0100': 2, '1000': 3})

# Show the first few rows of the data frame
# data.head()
data.iloc[:, start_index:end_index].head()


,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread
0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,1,0,1,1,0,0,0,1,0,...,1,2,1,0,1,0,2,2,0,2
3,0,1,0,2,2,2,1,1,2,0,...,2,2,1,0,0,0,2,2,0,2
4,0,2,0,0,1,0,1,1,2,1,...,1,2,2,1,1,0,2,2,0,2


In [26]:
data['Q3: AGE'].median()

41.0

# Column 111?

In [27]:
COLUMN_NAME = "Q10: DRESS"

def hot_encoding_values(df, col):
    nominal_values = df[col].unique().tolist()
    
    encoding_vals = {elem: idx for idx, elem in enumerate(nominal_values)}
    # can return the hashmap to see exactly what pertains to what as well. 
    df[col].replace(encoding_vals, inplace=True)
    print(df[col])
    

hot_encoding_values(data, COLUMN_NAME)

0       0
1       0
2       1
3       0
4       2
       ..
2455    0
2456    0
2457    2
2458    0
2459    0
Name: Q10: DRESS, Length: 2346, dtype: int64


# Column 112
- ### Oppenheimer this column 💣

In [28]:
# drop the empty cell column. 
data.drop(data.columns[112], axis=1, inplace=True)
data.iloc[:, 112:113].head()

,Q11: DAY
0,NaN
1,NaN
2,Friday
3,NaN
4,Sunday


# Column 112 

- ### This column is now 112, in the OG set it is 113. However we dropped an empty column with no useful information above

In [29]:
# These are nominal values as well so we can in theory apply hot encoding to these values pertaining to the date
COLUMN_NAME = "Q11: DAY"
COLUMN_NAME = data.columns[112]
print(COLUMN_NAME)

    
    
# encode the vals
hot_encoding_values(data, COLUMN_NAME)

Q11: DAY
0       0
1       0
2       1
3       0
4       2
       ..
2455    0
2456    0
2457    1
2458    0
2459    1
Name: Q11: DAY, Length: 2346, dtype: int64
